Importing the necessary libraries

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

Declaring all the necessary functions

In [2]:
def star(star_rating):
    l1,max=[],0
    stars=star_rating.findNext()
    star=stars.find_all('span',class_='star fill')
    for j in range (len(star)):#Creating a star list 
        l1.append(star[j].text.strip())
        l1[j]=int(l1[j])
    for i in range(len(l1)):# finding the maximum value
        if l1[i]>max:
            max=l1[i]
        else:
            continue
    return max
def starmaker(food_and_beverages,food): 
    if food is None:
        food_and_beverages.append('NA')
    else:
        food_and_beverages.append(star(food))       
def starscraper(review_block,block,seat_comfort,cabin_staff_service,food_and_beverages,inflight_entertainment,ground_service,value_for_money,wifi_and_connectivity):
    #food_and_beverage
    food=review_block[block].find('td',class_="review-rating-header food_and_beverages")
    starmaker(food_and_beverages,food)
    #seat comfort
    seat=review_block[block].find('td',class_='review-rating-header seat_comfort')
    starmaker(seat_comfort,seat)
    #ground service
    ground=review_block[block].find('td',class_='review-rating-header ground_service')
    starmaker(ground_service,ground)   
    #cabin_stff_service
    cabin_staff=review_block[block].find('td',class_="review-rating-header cabin_staff_service")
    starmaker(cabin_staff_service,cabin_staff)
    #value for money 
    value_money=review_block[block].find('td',class_='review-rating-header value_for_money')
    starmaker(value_for_money,value_money)
    #inflight entertainment
    flight_entertainment=review_block[block].find('td',class_='review-rating-header inflight_entertainment')
    starmaker(inflight_entertainment,flight_entertainment)
    #wifi_and_connectivity
    wifi_connectivity=review_block[block].find('td',class_='review-rating-header wifi_and_connectivity')
    starmaker(wifi_and_connectivity,wifi_connectivity)

In [3]:
def text_scraper(seat_type,cabin_type):
    if seat_type is None:
        cabin_type.append('NA')
    else:
        seat_type=seat_type.findNext().text.strip()
        cabin_type.append(seat_type)
def router(review_block,block,arrival,departure,via):
    route=review_block[block].find('td',class_='review-rating-header route')
    if route is None:
        arrival.append('NA')
        departure.append('NA')
        via.append('NA')
    else:
        route=route.findNext().text.strip()
        string=list(route.partition('to'))
        departure.append(string[0])
        str='via'
        if str in string[2]:
            splitter=string[2].partition('via')
            arrival.append(splitter[0])
            via.append(splitter[2])
        else:
            arrival.append(string[2])
            via.append('NA')
        # else:
        #     departure.append(Route[1])
        #     via.append('NA')
def classifier(review_block,i,cabin_type,date_of_travel,type_of_traveller,aircraft_type):
    aircraft=review_block[i].find('td',class_='review-rating-header aircraft')
    text_scraper(aircraft,aircraft_type)
    seat=review_block[i].find('td',class_='review-rating-header cabin_flown')
    text_scraper(seat,cabin_type)
    date_flown=review_block[i].find('td',class_="review-rating-header date_flown")
    text_scraper(date_flown,date_of_travel)
    traveller_type=review_block[i].find('td',class_='review-rating-header type_of_traveller')
    text_scraper(traveller_type,type_of_traveller)
def recommend(review_block,i,recommended):
    recom=review_block[i].find('td',class_='review-rating-header recommended')
    if recom is None:
        recommended.append('NA')
    else:
        recommendation=review_block[i].find('td',class_='review-value rating-no')
        if recommendation is None:
            recommended.append('YES')
        else:
            recommended.append('NO')
def date_division(date_of_travel,block,month_flown,year_flown):
    if date_of_travel[block] =='NA' or date_of_travel[block] is None:
        print('')
    else:
        # for i in range(len(date_of_travel)):
        k=date_of_travel[block].partition(' ')
        month_flown.append(k[0])
        year_flown.append(k[2])
def date_div(date_published,block,month_published,year_published):
    splitter=date_published[block].split()
    month_published.append(splitter[1])
    year_published.append(splitter[2])

In [4]:
def numeric(overall_rating):
    for i in range(len(overall_rating)):
        overall_rating[i]=int(overall_rating[i].text.strip())
def alphabetic(verified):
    for i in range(len(verified)):
        verified[i]=verified[i].text.strip()
    verified=list(verified)

Declaring all the star lists

In [5]:
seat_comfort,cabin_staff_service,food_and_beverages,inflight_entertainment,ground_service,value_for_money,wifi_and_connectivity,recommended=[],[],[],[],[],[],[],[]

Declaring all the text list

In [6]:
month_flown,year_flown,cabin_type,date_of_travel,arrival,departure,via,type_of_traveller,recommended,aircraft_type=[],[],[],[],[],[],[],[],[],[]

Sending request to server

In [7]:
for link in range(1,3):
    url=f"https://www.airlinequality.com/airline-reviews/british-airways/page/{link}/?sortby=post_date%3ADesc&pagesize=100"
    r=requests.get(url)

    # Creating the html document using lxml parser
    soup=BeautifulSoup(r.text,'lxml')
    soup.prettify()

    topic=soup.find_all('h2',class_='text_header')
    alphabetic(topic)

    # Check whether the information is verified or not 
    verified=soup.find_all("em")
    alphabetic(verified)

    # Name of passenger
    name_of_passenger=soup.find_all('span',itemprop='name')
    alphabetic(name_of_passenger)

    # Month and year of publication
    month_published,year_published=[],[]
    date_published=soup.find_all('time',itemprop='datePublished')
    alphabetic(date_published)
    for i in range(len(date_published)):
        date_div(date_published,i,month_published,year_published)

    # Overall rating by customer
    overall_rating=soup.find_all("span",itemprop="ratingValue")
    numeric(overall_rating)
    del overall_rating[0]
    overall_rating=list(overall_rating)

    # Remarks by customer
    remark=soup.find_all("div",class_="text_content",itemprop="reviewBody")
    alphabetic(remark)
    for i in range(len(remark)):
        k=remark[i].split('|')
        del k[0]
        remark[i]=k
    remark=list(remark)

    # The below block contains all the information of passenger and their reviews
    review_block=soup.find_all("table",class_="review-ratings")
    del review_block[0]
    review_block=list(review_block)

    # Text based scripting
    for block in range(len(review_block)):
        starscraper(review_block,block,seat_comfort,cabin_staff_service,food_and_beverages,inflight_entertainment,ground_service,value_for_money,wifi_and_connectivity)
        classifier(review_block,block,cabin_type,date_of_travel,type_of_traveller,aircraft_type)
        router(review_block,block,arrival,departure,via)
        recommend(review_block,block,recommended)
        date_division(date_of_travel,block,month_flown,year_flown)
    print(f"Page number {link} completed")
# DataFrame
British_Airways=pd.DataFrame({'Name of Passenger':name_of_passenger,'Verified':verified,'Heading':topic,'Remarks':remark,'Overall Rating':overall_rating,'Aircraft Type':aircraft_type,'Arrival':arrival,'Departure':departure,'Via Route':via,'Seat Type':cabin_type,'Traveller Type':type_of_traveller,'Month Flown':month_flown,'Year Flown':year_flown,'Review Published (Month)':month_published,'Review Published (Year)':year_published,'Cabin Staff Service':cabin_staff_service,'Food and Beverage':food_and_beverages,'Wifi and connectivity':wifi_and_connectivity,'Ground Service':ground_service,'Seat Comfort':seat_comfort,'Value for meny':value_for_money,'Inflight Entertainment':inflight_entertainment,'Recommended':recommended})

Page number 1 completed
Page number 2 completed


In [ ]:
# British_Airways

,Name of Passenger,Verified,Heading,Remarks,Overall Rating,Aircraft Type,Arrival,Departure,Via Route,Seat Type,...,Review Published (Month),Review Published (Year),Cabin Staff Service,Food and Beverage,Wifi and connectivity,Ground Service,Seat Comfort,Value for meny,Inflight Entertainment,Recommended
0,Carol Jones,Trip Verified,"""I will not be flying BA again""",[ Manchester to Seattle via London. I will ne...,4,NA,Seattle,Manchester,London,Economy Class,...,August,2022,2,1,NA,4,3,3,2,NO
1,E Garton,Trip Verified,"""not been recompensed for our expenses""",[ Our flight in March 2022 from San Francisco...,1,NA,London,San Francisco,NA,Premium Economy,...,August,2022,1,2,NA,1,2,1,3,NO
2,Alison Green,Not Verified,"""she went above and beyond""",[ I sat in seat 25C. Our Cabin Crew lady was ...,10,NA,London City,Berlin,NA,Economy Class,...,August,2022,5,4,NA,2,3,4,NA,YES
3,Neil Nerva,Trip Verified,"""Hard working crew""",[ Full afternoon flight. Ready to fly on time...,9,Boeing 787,London,Tel Aviv,NA,Economy Class,...,August,2022,5,5,NA,3,5,4,5,YES
4,Raj Gharu,Trip Verified,"""Philippines Airlines require a 24 hr negative...",[ I was told I'd require a 48 hr negative cov...,1,NA,Manila,London,Singapore,Business Class,...,August,2022,1,1,NA,1,1,1,1,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Chris Walsh,Trip Verified,"""Service was good but food not so""",[ Very small plane for a busy route. Service w...,7,A319,Venice,London,NA,Business Class,...,April,2022,4,2,NA,2,2,3,NA,YES
96,G Meaden,Trip Verified,"""still no refunds 3 months on""",[ BA continue to charge a premium for the mos...,1,A320,Barcelona,London,NA,Business Class,...,April,2022,1,1,NA,1,1,1,NA,NO
97,Richard Hodges,Trip Verified,"""old Club Class a huge disappointment""",[ Flew back on BA0254 Bridgetown to Heathrow ...,5,Boeing 777,Heathrow,Barbados,NA,Business Class,...,April,2022,3,3,NA,4,2,3,1,YES
98,Haydn Robinson,Not Verified,"""Spent an hour checking in""",[ The worst budget airline I’ve ever used! Fa...,1,NA,Bologna,Heathrow,NA,Economy Class,...,April,2022,1,NA,NA,1,1,1,NA,NO
